# 0. Imports

In [1]:
!pip install pyDes

  Preparing metadata (setup.py) ... done
  Created wheel for pyDes: filename=pyDes-2.0.1-py2.py3-none-any.whl size=9555 sha256=e46836e2d2f4df40ff59ed7994f633b32c752375c6946fbfa959386e9cf7c5f4
  Stored in directory: /root/.cache/pip/wheels/30/04/38/5cb6e4f33d5c481aa1c213e9c1763533e18c717e964bde5bee
Successfully built pyDes


In [2]:
from pyDes import des, PAD_PKCS5
import binascii
from collections import deque

# 1. Implementation

In [3]:
# Implementation of the algorithm


initial_permutation_list =   [58, 50, 42, 34, 26, 18, 10, 2,
                              60, 52, 44, 36, 28, 20, 12, 4,
                              62, 54, 46, 38, 30, 22, 14, 6,
                              64, 56, 48, 40, 32, 24, 16, 8,
                              57, 49, 41, 33, 25, 17, 9,  1,
                              59, 51, 43, 35, 27, 19, 11, 3,
                              61, 53, 45, 37, 29, 21, 13, 5,
                              63, 55, 47, 39, 31, 23, 15, 7]

final_permutation_list = [40, 8, 48, 16, 56, 24, 64, 32,
                          39, 7, 47, 15, 55, 23, 63, 31,
                          38, 6, 46, 14, 54, 22, 62, 30,
                          37, 5, 45, 13, 53, 21, 61, 29,
                          36, 4, 44, 12, 52, 20, 60, 28,
                          35, 3, 43, 11, 51, 19, 59, 27,
                          34, 2, 42, 10, 50, 18, 58, 26,
                          33, 1, 41, 9,  49, 17, 57, 25]

E_permutation_list = [32, 1,  2,  3,  4,  5,
                      4,  5,  6,  7,  8,  9,
                      8,  9,  10, 11, 12, 13,
                      12, 13, 14, 15, 16, 17,
                      16, 17, 18, 19, 20, 21,
                      20, 21, 22, 23, 24, 25,
                      24, 25, 26, 27, 28, 29,
                      28, 29, 30, 31, 32, 1]

P_permutation_list = [16,  7, 20, 21, 29, 12, 28, 17,
                       1, 15, 23, 26,  5, 18, 31, 10,
                       2,  8, 24, 14, 32, 27,  3,  9,
                      19, 13, 30,  6, 22, 11,  4, 25]

S1_matrix = [ [14, 4,  13, 1, 2 , 15, 11, 8, 3,  10, 6, 12, 5, 9, 0,  7],
              [0,  15, 7,  4, 14, 2,  13, 1, 10, 6, 12, 11, 9, 5, 3,  8],
              [4,  1,  14, 8, 13, 6,  2, 11, 15, 12, 9, 7,  3, 10, 5, 0],
              [15, 12, 8,  2, 4,  9,  1, 7,  5,  11, 3, 14, 10, 0, 6, 13] ]

S2_matrix = [ [15, 1,  8,  14, 6,  11, 3,  4,  9,  7, 2,  13, 12, 0, 5,  10],
              [3,  13, 4,  7,  15, 2,  8,  14, 12, 0, 1,  10, 6,  9, 11, 5],
              [0,  14, 7,  11, 10, 4,  13, 1,  5,  8, 12, 6,  9,  3, 2,  15],
              [13, 8,  10, 1,  3,  15, 4,  2,  11, 6, 7,  12, 0,  5, 14, 9]]

S3_matrix = [ [10, 0, 9, 14, 6, 3,  15, 5,  1,  13, 12, 7,  11, 4,  2,  8],
              [13, 7, 0, 9,  3, 4,  6,  10, 2,  8,  5,  14, 12, 11, 15, 1],
              [13, 6, 4, 9,  8, 15, 3,  0,  11, 1,  2,  12, 5,  10, 14, 7],
              [1, 10, 13, 0, 6, 9,  8,  7,  4,  15, 14, 3,  11, 5,  2,  12]]

S4_matrix = [ [7,  13,  14, 3,  0,  6,  9, 10,  1,  2,  8,  5, 11, 12,  4, 15],
              [13, 8,   11, 5,  6, 15,  0,  3,  4,  7,  2, 12,  1, 10, 14,  9],
              [10, 6,   9,  0, 12, 11,  7, 13, 15,  1,  3, 14,  5,  2,  8,  4],
              [3,  15,  0,  6, 10,  1,  13, 8,  9,  4,  5, 11, 12,  7,  2, 14]]

S5_matrix = [ [2,  12,  4,  1,  7, 10, 11,  6,  8,  5,  3, 15, 13,  0, 14,  9],
              [14, 11,  2, 12,  4,  7, 13,  1,  5,  0, 15, 10,  3,  9,  8,  6],
              [4,  2,   1, 11, 10, 13,  7,  8, 15,  9, 12,  5,  6,  3,  0, 14],
              [11, 8,  12,  7,  1, 14,  2, 13,  6, 15,  0,  9, 10,  4,  5,  3]]

S6_matrix = [ [12,  1, 10, 15,  9,  2,  6,  8,  0, 13,  3,  4, 14,  7,  5, 11],
              [10, 15,  4,  2,  7, 12,  9,  5,  6,  1, 13, 14,  0, 11,  3,  8],
              [ 9, 14, 15,  5,  2,  8, 12,  3,  7,  0,  4, 10,  1, 13, 11,  6],
              [ 4,  3,  2, 12,  9,  5, 15, 10, 11, 14,  1,  7,  6,  0,  8, 13]]

S7_matrix = [ [ 4, 11,  2, 14, 15,  0,  8, 13,  3, 12,  9,  7,  5, 10,  6,  1],
              [13,  0, 11,  7,  4,  9,  1, 10, 14,  3,  5, 12,  2, 15,  8,  6],
              [ 1,  4, 11, 13, 12,  3,  7, 14, 10, 15,  6,  8,  0,  5,  9,  2],
              [ 6, 11, 13,  8,  1,  4, 10,  7,  9,  5,  0, 15, 14,  2,  3, 12]]

S8_matrix = [ [13,  2,  8,  4,  6, 15, 11,  1, 10,  9,  3, 14,  5,  0, 12,  7],
              [ 1, 15, 13,  8, 10,  3,  7,  4, 12,  5,  6, 11,  0, 14,  9,  2],
              [ 7, 11,  4,  1,  9, 12, 14,  2,  0,  6, 10, 13, 15,  3,  5,  8],
              [ 2,  1, 14,  7,  4, 10,  8, 13, 15, 12,  9,  0,  3,  5,  6, 11]]

PC1_permutation_list =   [57, 49, 41, 33, 25, 17,  9,  1,
                          58, 50, 42, 34, 26, 18, 10,  2,
                          59, 51, 43, 35, 27, 19, 11,  3,
                          60, 52, 44, 36, 63, 55, 47, 39,
                          31, 23, 15,  7, 62, 54, 46, 38,
                          30, 22, 14,  6, 61, 53, 45, 37,
                          29, 21, 13,  5, 28, 20, 12,  4]

PC2_permitation_list = [14, 17, 11, 24,  1,  5,  3, 28,
                        15,  6, 21, 10, 23, 19, 12,  4,
                        26,  8, 16,  7, 27, 20, 13,  2,
                        41, 52, 31, 37, 47, 55, 30, 40,
                        51, 45, 33, 48, 44, 49, 39, 56,
                        34, 53, 46, 42, 50, 36, 29, 32]


hex_to_bitlist = {'0': [0, 0, 0, 0],
                  '1': [0, 0, 0, 1],
                  '2': [0, 0, 1, 0],
                  '3': [0, 0, 1, 1],
                  '4': [0, 1, 0, 0],
                  '5': [0, 1, 0, 1],
                  '6': [0, 1, 1, 0],
                  '7': [0, 1, 1, 1],
                  '8': [1, 0, 0, 0],
                  '9': [1, 0, 0, 1],
                  'A': [1, 0, 1, 0],
                  'B': [1, 0, 1, 1],
                  'C': [1, 1, 0, 0],
                  'D': [1, 1, 0, 1],
                  'E': [1, 1, 1, 0],
                  'F': [1, 1, 1, 1]}

bits_to_hex =    {  '0000' : '0',
                    '0001' : '1',
                    '0010' : '2',
                    '0011' : '3',
                    '0100' : '4',
                    '0101' : '5',
                    '0110' : '6',
                    '0111' : '7',
                    '1000' : '8',
                    '1001' : '9',
                    '1010' : 'A',
                    '1011' : 'B',
                    '1100' : 'C',
                    '1101' : 'D',
                    '1110' : 'E',
                    '1111' : 'F'}


def int_to_bit_list(int_number, bit_list_len):

    bits_list = deque()

    for i in range(0, bit_list_len):

        bits_list.appendleft(min(int_number & (1 << i), 1))

    return list(bits_list)


def permute(bit_list : list, permutation_list : list, new_len) -> list:


    # create a new zero bit string
    permuted_bit_list = [0] * new_len

    # permute input string bits into new bit string
    for i in range(new_len):

        # index to be exchanged
        j = permutation_list[i] - 1

        # exchange
        permuted_bit_list[i] = bit_list[j]

    return permuted_bit_list


def initial_permutation(bit_list : list) -> list:
    return permute(bit_list, initial_permutation_list, 64)


def expansion_permutation(bit_list : list) -> list:
    return permute(bit_list, E_permutation_list, 48)

def P_permutation(bit_list : list) -> list:
    return permute(bit_list, P_permutation_list, 32)

def PC1_permutation(bit_list : list) -> list:
    return permute(bit_list, PC1_permutation_list, 56)

def PC2_permutation(bit_list : list) -> list:
    return permute(bit_list, PC2_permitation_list, 48)

def final_permutation(bit_list):
    return permute(bit_list, final_permutation_list, 64)


def xor(bit_list1 : list, bit_list2 : list) -> list:

    xord_list = [0]*len(bit_list1)

    for i in range(len(bit_list1)):
        xord_list[i] = bit_list1[i] ^ bit_list2[i]

    return xord_list

def sbox(bit_list, S_matrix):

    # get coordinates of output in s-box
    row = bit_list[0]*2 + bit_list[-1]*1
    col = bit_list[1]*8 + bit_list[2]*4 + bit_list[3]*2 + bit_list[4]*1

    # get output by indexing the s-box
    out = S_matrix[row][col]

    # return a binary list representation of the number
    return int_to_bit_list(out, 4)

def f_function(R0_list, key):

    # expansion
    expanded_list = expansion_permutation(R0_list)

    # xor with key
    xord_list = xor(expanded_list, key)

    # S1
    S1_out = sbox(xord_list[:6],    S1_matrix)

    # S2
    S2_out = sbox(xord_list[6:12],  S2_matrix)

    # S3
    S3_out = sbox(xord_list[12:18], S3_matrix)

    # S4
    S4_out = sbox(xord_list[18:24], S4_matrix)

    # S5
    S5_out = sbox(xord_list[24:30], S5_matrix)

    # S6
    S6_out = sbox(xord_list[30:36], S6_matrix)

    # S7
    S7_out = sbox(xord_list[36:42], S7_matrix)

    # S8
    S8_out = sbox(xord_list[42:48], S8_matrix)

    # P Permutation block
    out = P_permutation(S1_out + S2_out + S3_out + S4_out + S5_out + S6_out + S7_out + S8_out)

    return out

def key_transform(bit_list, i):

    # divide into two halves
    l = bit_list[:len(bit_list)//2]
    r = bit_list[len(bit_list)//2:]

    # rotate each half to left by 1
    l.append(l.pop(0))
    r.append(r.pop(0))

    # one extra left rotation for i = 1, 2, 9, 16 (1 based indexing)
    if i not in [1, 2, 9, 16]:
        l.append(l.pop(0))
        r.append(r.pop(0))

    # save the 56 bit key
    key_56 = l + r

    # permute by PC2 to get the 48 bit key
    key_48 = PC2_permutation(key_56)

    return key_48, key_56

def roundi(bit_list, key_48):

    # divide input list into two halves
    l = bit_list[:len(bit_list)//2]
    r = bit_list[len(bit_list)//2:]

    # use the f function on the right half and the key
    f_out = f_function(r, key_48)

    # xor the output of f function with left half
    xor_out = xor(l, f_out)

    # group halves together
    return r + xor_out


def hex_string_to_bit_list(hex_string):

    # initialize the output bit list
    bit_list = []

    # convert each hexadecimal character in input string to bit list and append to the main list
    for c in hex_string:
        bit_list.extend(hex_to_bitlist[c])

    return bit_list

def bit_list_to_hex_string(bit_list):


    hex_string = ""

    # loop over all characters
    for i in range(len(bit_list)//4):

        bit_string = ''
        # loop over bits of a single character
        for j in range(4):

            bit_string = bit_string + str(bit_list[i*4+j])

        hex_string = hex_string + bits_to_hex[bit_string]

    return hex_string




def DES_encrypt(plaintext, key_64):

    # change the plaintext into a list of bits
    plaintext = hex_string_to_bit_list(plaintext)

    # change the key into a list of bits
    key_64 = hex_string_to_bit_list(key_64)

    # apply initial permutation (IP) to the plain list of bits
    IP = initial_permutation(plaintext)

    # apply PC-1 to the initial 64 bit key
    key_56 = PC1_permutation(key_64)

    # loop for a total of 16 rounds
    keyi_56 = key_56
    IPi = IP
    for i in range(1, 17):

        # generate the i-th subkey
        keyi_48, keyi_56 = key_transform(keyi_56, i)

        # carry out calculations of the ith round
        IPi = roundi(IPi, keyi_48)

    # switch L and R of the ciphertext one last time
    IPi[:len(IPi)//2], IPi[len(IPi)//2:] = IPi[len(IPi)//2:], IPi[:len(IPi)//2]

    # perform the final permutation
    fIP = final_permutation(IPi)

    # transform the list of bits into hex again
    ciphertext = bit_list_to_hex_string(fIP)

    return ciphertext

# 2. Testing

In [4]:
hex_key = "266200199BBCDFF1"
hex_input = "0123456789ABCDEF"

DES_encrypt(hex_input, hex_key)

'4E0E6864B5E1CA52'

In [6]:
# Comparison with PyDes


hex_key = "266200199BBCDFE1"
hex_input = "0123466789ABCDEF"

hex_key = "0000000000000000"
hex_input = "0000000000000000"

byte_key = bytes.fromhex(hex_key)
byte_input = bytes.fromhex(hex_input)


cipher = des(byte_key)

encrypted_text = cipher.encrypt(byte_input)


encrypted_hex = binascii.hexlify(encrypted_text).decode('utf-8')

print(f"PyDes: {encrypted_hex.upper()}")

print(f"Implemented: {DES_encrypt(hex_input, hex_key)}")


PyDes: 8CA64DE9C1B123A7
Implemented: 8CA64DE9C1B123A7
